In [0]:
# core imports
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
import seaborn as sns
import itertools
import os

from sklearn.model_selection import GroupKFold
from sklearn.preprocessing import MinMaxScaler

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Masking
from keras.layers import Dropout
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

##### Importing the dataset

In [0]:
# the datasets are pickle files in the Data_LSTM_Analysis folder
# get the relative path to the dataset
fileDir = os.path.dirname(os.path.abspath(__file__))
parentDir = os.path.dirname(fileDir)
targetDir = os.path.join(parentDir, "Datasets", "Data_LSTM_Analysis")

datapath = targetDir + "/"

# open a saved dictionary
def open_dict(name):
    with open(datapath + name + ".pkl", "rb") as f:
        return pickle.load(f)

<h2>Time Series Classification using a Long-Short-Term-Memory Neural Newtork</h2>
<h3>Inspired by Blog Posts on https://machinelearningmastery.com/</h3>
<p>More specifically:</p>
<ul>
<li>https://machinelearningmastery.com/sequence-classification-lstm-recurrent-neural-networks-python-keras/</li>
<li>https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/</li>
<li>https://machinelearningmastery.com/prepare-univariate-time-series-data-long-short-term-memory-networks/</li>
<li>https://machinelearningmastery.com/sequence-prediction/</li>
</ul>


<p>General Instruction to implement to LSTM for time-series classification using mouse usage data</p>

* Interpolation of mouse movement data into equal timesteps
* If applicable, seperate the mouse data into data per trial with overlaps at the beginning and end of a trial
* Set the input lenght of to be between 200 and 400 data points (according to recommendations in the blog)
* All trial datasets (or other subdatasets need to be of equal length, which requires trimming the datasets or filling the datastes with 0s)
* Train a simple LSTM and test the prediction accuracy







In [0]:
# Import the datasets

pointclick_data = open_dict("Data_PointClick_Task")
dragdrop_data = open_dict("Data_DragDrop_Task")
followbox_data = open_dict("Data_Drawing_Task")
drawing_data = open_dict("Data_FollowBox_Task")

# data structure:
# {participant: 
#   {high_stress: 
#     {circlesClicked: [...], eventtype[...], time: [...], x: [...], y: [...], DiffTime: [...]}, 
#    low_stress:
#       {circlesClicked: [...], eventtype[...], time: [...], x: [...], y: [...], DiffTime: [...]}}}

In [0]:
# Playing around to understand the data

for i in followbox_data:
  for k in followbox_data[i]:
    for l in followbox_data[i][k]:
      print(l)
    break
  break

inBox
eventType
time
x
y
DiffTime


<h3>Reusable functions</h3>

In [0]:
# interpolate the mouse data
def interpolate_mouse_data(coordinate, time):

    interp = interpolate.interp1d(time, coordinate)

    # set start and end point of new timeline
    start = time[0]
    end = time[-1]

    # create a new timeline array with equal timesteps using the start and endpoints
    new_times = np.arange(start, end, 15)

    # use the interpolation function to calculate the interpolated x- and y-coordinates on the equally spaced time
    # interval 
    new_coord = np.round(interp(new_times), decimals=3)

    # visualize the original vs the interpolated data)
    # plt.plot(new_times, new_coord, linestyle="--")
    # plt.plot(time, coordinate, linestyle=":")
    #
    # plt.show()

    return (new_coord, new_times)

In [0]:
# split a list into parts based on an indexlist
# Splits the list WITH OVERLAP
# useful for all tasks that have a "trial structure"
def sep_list(separator, target_list):
  
  separated_list = []
  
  startpoint = 0
  
  for pos, ind in enumerate(separator):
    
    endpoint = startpoint + ind
    
    # splits the list based on the starting and end point plus 5 more datapoints
    # if there are enough datapoints in the prev and following trial
    
    # the first trial can only have overlap with the next but not previous trial
    if pos == 0:
      len_next = 5 if separator[pos + 1] >= 5 else separator[pos + 1]
      separated_list.append(target_list[startpoint:endpoint + len_next])
    # the last trial can only have overlap with the previous but not next trial
    elif pos == len(separator) - 1:
      len_prev = 5 if separator[pos - 1] >= 5 else separator[pos - 1]
      separated_list.append(target_list[startpoint - len_prev:endpoint])
    else:
      len_next = 5 if separator[pos + 1] >= 5 else separator[pos + 1]
      len_prev = 5 if separator[pos - 1] >= 5 else separator[pos - 1]
      separated_list.append(target_list[startpoint - len_prev:endpoint + len_next])
      
    startpoint = endpoint
    
  return separated_list
    

<h3>Point and Click Dataset Creation</h3>

In [0]:
# group the Data of each participant into trials

def create_point_click_data():

  # Prepare observations/examples
  input_series = []
  output_classes = []
  groups = []

  bad_cases_point_click = ["INFvFzlNN7UeZRqvtoe8hejtnvX2",
                              "PRPyIVrPUVgCFtZ60nqvXc4wUov2",
                              "Vhi1KMa2MecuTnTzWb8QBp9rNDw1",
                              "X5mfB1vgISTp6cagPLnp0zyh4EI2",
                              "ZnI96HDrRZV9xNlfQxgTU0CeoDX2",
                              "zf3H80XYGSf9U3pZM5xxOm69qTT2"]

  # add emtpy dictionaries to bad cases (participant did not provide data)
  for participant in pointclick_data:
    if not pointclick_data[participant]:
      bad_cases_point_click.append(participant)
  
  
  # delete the bad cases from the dataset
  for participant in bad_cases_point_click:
    pointclick_data.pop(participant, None)

# loop over the "good data"  
for par_num, participant in enumerate(pointclick_data):
    

    # create a participant dummy variable to store information about the
    # participant
    par_dummy = [1 if i == par_num else 0 for i in range(len(pointclick_data))]
    
    # loop over the tasks conditions (high-stress or low-stress)
    for task in pointclick_data[participant]:
        
        # save the task outcome
        if "HS" in task:
          output = 1
        elif "LS" in task:
          output = 0
        
        # normalize the x & y data using the screen width & height
        x_coords = np.asarray(pointclick_data[participant][task]["x"]) / 1920
        y_coords = np.asarray(pointclick_data[participant][task]["y"]) / 1080
        
        # separate the lists containing all data into a list with trial data
        trial_separator = [len(list(y)) for x,y in itertools.groupby(pointclick_data[participant][task]["circlesClicked"])]
        
        sep_x = sep_list(trial_separator, x_coords)
        sep_y = sep_list(trial_separator, y_coords)
        sep_time = sep_list(trial_separator, pointclick_data[participant][task]["DiffTime"])
        sep_eventType = sep_list(trial_separator, pointclick_data[participant][task]["eventType"])
        
        # interpolate x, y and time
        
        int_x = [interpolate_mouse_data(sep_x[i], sep_time[i])[0] for i in range(len(sep_x))]
        int_y = [interpolate_mouse_data(sep_y[i], sep_time[i])[0] for i in range(len(sep_y))]
        int_time = [interpolate_mouse_data(sep_y[i], sep_time[i])[1] for i in range(len(sep_y))]
        
        # create a list with the timings for a click event in a trial
        all_clicktimes = []
        # loop over the eventtype list
        for i, trial in enumerate(sep_eventType):
          trial_clickTimes = []
          for k, event in enumerate(trial):
            # if the eventtype is a mouseclick, save the corresponding timestamp
            if event == "MouseClick":
              trial_clickTimes.append(sep_time[i][k])
          all_clicktimes.append(trial_clickTimes)
              
        # create a list with the indices in the interpolated lists that represent
        # mouseclicks
        click_index = []
        # loop over all times in all trials
        for ind, trial in enumerate(all_clicktimes):
          trial_index = []
          for clicktime in trial:
            # find the first entry of the smallest difference between the original
            # click timestamp and the timestamps in the interpolated list
            click_ind = np.where(np.abs(clicktime - int_time[ind]) == np.ndarray.min(np.abs(clicktime - int_time[ind])))[0][0]
            trial_index.append(click_ind)
          click_index.append(trial_index)
            
        # create an event type list that stores information about the eventType
        # of the interpolated datapoint
        new_eventType = []
        for num, trial in enumerate(click_index):
          events = [0 if i in trial else 1 for i in range(len(int_time[num]))]
          new_eventType.append(events)
        
        # add every trial as a feature to the input series list
        for num in range(len(int_x)):
          
          # store all features in a list and also add the participant dummy
          features = [int_x[num], int_y[num], new_eventType[num], par_dummy]
          # append the features and the output per trial
          input_series.append(features)
          output_classes.append(output)
          groups.append(par_num)
    
  return input_series, output_classes, groups
      

<h3>Drag and Drop Task Dataset Creation</h3>

In [0]:
# group the Data of each participant into trials

def create_drag_drop_data():

  # Prepare observations/examples
  input_series = []
  output_classes = []
  groups = []

  bad_cases_drag_drop = [
         "Vhi1KMa2MecuTnTzWb8QBp9rNDw1",
        "ZnI96HDrRZV9xNlfQxgTU0CeoDX2",
        "dOXefJ4hHEfRsYTTFfn6NfATnh32"]

  # add emtpy dictionaries to bad cases (participant did not provide data)
  for participant in dragdrop_data:
    if not dragdrop_data[participant]:
      bad_cases_drag_drop.append(participant)

  # delete the bad cases from the dataset
  for participant in bad_cases_drag_drop:
    dragdrop_data.pop(participant, None)

  # information about drag and drop task
  for par_num, participant in enumerate(dragdrop_data):
    

    # create a participant dummy variable
    par_dummy = [1 if i == par_num else 0 for i in range(len(dragdrop_data))]
    
    # loop over the tasks conditions (high-stress or low-stress)
    for task in dragdrop_data[participant]:
        
        # save the task outcome
        if "HS" in task:
          output = 1
        elif "LS" in task:
          output = 0
        
        # normalize the x & y data using the screen width & height
        x_coords = np.asarray(dragdrop_data[participant][task]["x"]) / 1920
        y_coords = np.asarray(dragdrop_data[participant][task]["y"]) / 1080
        
        # separate the lists containing all data into a list with trial data
        trial_separator = [len(list(y)) for x,y in itertools.groupby(dragdrop_data[participant][task]["circlesDragged"])]
        
        sep_x = sep_list(trial_separator, x_coords)
        sep_y = sep_list(trial_separator, y_coords)
        sep_time = sep_list(trial_separator, dragdrop_data[participant][task]["DiffTime"])
        sep_eventType = sep_list(trial_separator, dragdrop_data[participant][task]["eventType"])
        
        # interpolate x, y and time
        
        int_x = [interpolate_mouse_data(sep_x[i], sep_time[i])[0] for i in range(len(sep_x))]
        int_y = [interpolate_mouse_data(sep_y[i], sep_time[i])[0] for i in range(len(sep_y))]
        int_time = [interpolate_mouse_data(sep_y[i], sep_time[i])[1] for i in range(len(sep_y))]
        
        # create a list with the timings for a click event in a trial
        all_clicktimes = []
        # loop over the eventtype list
        for i, trial in enumerate(sep_eventType):
          trial_clickTimes = []
          for k, event in enumerate(trial):
            # if the eventtype is a mouseclick, save the corresponding timestamp
            if event == "MouseClick":
              trial_clickTimes.append(sep_time[i][k])
          all_clicktimes.append(trial_clickTimes)
              
        # create a list with the indices in the interpolated lists that represent
        # mouseclicks
        click_index = []
        # loop over all times in all trials
        for ind, trial in enumerate(all_clicktimes):
          trial_index = []
          for clicktime in trial:
            # find the first entry of the smallest difference between the original
            # click timestamp and the timestamps in the interpolated list
            click_ind = np.where(np.abs(clicktime - int_time[ind]) == np.ndarray.min(np.abs(clicktime - int_time[ind])))[0][0]
            trial_index.append(click_ind)
          click_index.append(trial_index)
            
        # create an event type list that stores information about the eventType
        # of the interpolated datapoint
        new_eventType = []
        for num, trial in enumerate(click_index):
          events = [0 if i in trial else 1 for i in range(len(int_time[num]))]
          new_eventType.append(events)
        
         # add every trial as a feature to the input series list
        for num in range(len(int_x)):
          
          # store all features in a list and also add the participant dummy
          features = [int_x[num], int_y[num], new_eventType[num], par_dummy]
          # append the features and the output per trial
          input_series.append(features)
          output_classes.append(output)
          groups.append(par_num)
    
  return input_series, output_classes, groups
      

<h3>Drawing Task Dataset Creation</h3>

In [0]:
# group the Data of each participant into trials

def create_drawing_data():

  # Prepare observations/examples
  input_series = []
  output_classes = []
  groups = []

  bad_cases_drawing = [
         "4ibZh6HiVrPknUuvasf6CIVD1L42",
        "9DdqeM3mkRWvD4MCMcWvFH4wIhA2",
        "GzlieWRWOJgquA0nDZtBFExSW0E2",
        "PRPyIVrPUVgCFtZ60nqvXc4wUov2",
        "V6TlsMC21YbLX2MyjUiEiVrhgMk2",
        "Vhi1KMa2MecuTnTzWb8QBp9rNDw1",
        "WpwRDjuqqdRavEdFOgqbpRA19Q32",
        "b8KoZn71FhOsv46MKFAlqdhTBXM2",
        "bD83eNrrWTgjMkVAdibgZNiebTe2",
        "cgS7wdlEL3XTYUiBbXGDGCoyJWe2",
        "dOXefJ4hHEfRsYTTFfn6NfATnh32",
        "gJnYNGrs0lhMSKDh9i466E2MiJn2",
        "nDsahKaZCEUMI732rmmyANfauCj1",
        "nQl80EJnrxYnsuoVqEFhTMzZVgl2",
        "pPx9EQjAJuN5wxinUZjO6DUsZDo1",
        "uQ5ly8q3nPQZHi56WJFlw0Y3A4E2",
        "zoCupEvJwahE0hRlqKWLyP8TRrJ2"]

  # add emtpy dictionaries to bad cases (participant did not provide data)
  for participant in drawing_data:
    if not drawing_data[participant]:
      bad_cases_drawing.append(participant)

  # delete the bad cases from the dataset
  for participant in bad_cases_drawing:
    drawing_data.pop(participant, None)

  # information about drag and drop task
  for par_num, participant in enumerate(drawing_data):
    

    # create a participant dummy variable
    par_dummy = [1 if i == par_num else 0 for i in range(len(drawing_data))]
    

    for task in drawing_data[participant]:
        
        # save the task outcome
        if "HS" in task:
          output = 1
        elif "LS" in task:
          output = 0
        
        # normalize the x & y data using the screen width & height
        x_coords = np.asarray(drawing_data[participant][task]["x"]) / 1920
        y_coords = np.asarray(drawing_data[participant][task]["y"]) / 1080
        
        # separate the lists containing all data into a list with trial data
        trial_separator = [len(list(y)) for x,y in itertools.groupby(drawing_data[participant][task]["touchedMilestones"])]
        
        sep_x = sep_list(trial_separator, x_coords)
        sep_y = sep_list(trial_separator, y_coords)
        sep_time = sep_list(trial_separator, drawing_data[participant][task]["DiffTime"])
        sep_eventType = sep_list(trial_separator, drawing_data[participant][task]["eventType"])
        
        # interpolate x, y and time
        
        int_x = [interpolate_mouse_data(sep_x[i], sep_time[i])[0] for i in range(len(sep_x))]
        int_y = [interpolate_mouse_data(sep_y[i], sep_time[i])[0] for i in range(len(sep_y))]
        int_time = [interpolate_mouse_data(sep_y[i], sep_time[i])[1] for i in range(len(sep_y))]
        
        # create a list with the timings for a click event in a trial
        all_clicktimes = []
        # loop over the eventtype list
        for i, trial in enumerate(sep_eventType):
          trial_clickTimes = []
          for k, event in enumerate(trial):
            # if the eventtype is a mouseclick, save the corresponding timestamp
            if event == "MouseClick":
              trial_clickTimes.append(sep_time[i][k])
          all_clicktimes.append(trial_clickTimes)
              
        # create a list with the indices in the interpolated lists that represent
        # mouseclicks
        click_index = []
        # loop over all times in all trials
        for ind, trial in enumerate(all_clicktimes):
          trial_index = []
          for clicktime in trial:
            # find the first entry of the smallest difference between the original
            # click timestamp and the timestamps in the interpolated list
            click_ind = np.where(np.abs(clicktime - int_time[ind]) == np.ndarray.min(np.abs(clicktime - int_time[ind])))[0][0]
            trial_index.append(click_ind)
          click_index.append(trial_index)

        # create an event type list that stores information about the eventType
        # of the interpolated datapoint
        new_eventType = []
        for num, trial in enumerate(click_index):
          events = [0 if i in trial else 1 for i in range(len(int_time[num]))]
          new_eventType.append(events)
        
         # add every trial as a feature to the input series list
        for num in range(len(int_x)):
          
          # store all features in a list and also add the participant dummy
          features = [int_x[num], int_y[num], new_eventType[num], par_dummy]
          # append the features and the output per trial
          input_series.append(features)
          output_classes.append(output)
          groups.append(par_num)
    
  return input_series, output_classes, groups

<h3>Follow-Box Dataset Creation</h3>

In [0]:
# group the Data of each participant into trials

def create_follow_box_data():

  # Prepare observations/examples
  input_series = []
  output_classes = []
  groups = []

  bad_cases_box = [
        ]

  # add emtpy dictionaries to bad cases (participant did not provide data)
  for participant in followbox_data:
    if not followbox_data[participant]:
      bad_cases_box.append(participant)

  # delete the bad cases from the dataset
  for participant in bad_cases_box:
    followbox_data.pop(participant, None)

  # information about drag and drop task
  for par_num, participant in enumerate(followbox_data):
    

    # create a participant dummy variable
    par_dummy = [1 if i == par_num else 0 for i in range(len(followbox_data))]
    

    for task in followbox_data[participant]:
        
        # save the task outcome
        if "HS" in task:
          output = 1
        elif "LS" in task:
          output = 0
        
        # normalize the x & y data using the screen width & height
        x_coords = np.asarray(followbox_data[participant][task]["x"]) / 1920
        y_coords = np.asarray(followbox_data[participant][task]["y"]) / 1080
        
        # the follow box task cant be split by trials but has too many
        # datapoints --> the dataset is split into ruffly 5 equal parts
        sep_x = np.array_split(x_coords, 5)
        sep_y = np.array_split(y_coords, 5)
        sep_time = np.array_split(followbox_data[participant][task]["DiffTime"], 5)
        sep_eventType = np.array_split(followbox_data[participant][task]["eventType"], 5)
        
        # interpolate x, y and time
        
        int_x = [interpolate_mouse_data(sep_x[i], sep_time[i])[0] for i in range(len(sep_x))]
        int_y = [interpolate_mouse_data(sep_y[i], sep_time[i])[0] for i in range(len(sep_y))]
        int_time = [interpolate_mouse_data(sep_y[i], sep_time[i])[1] for i in range(len(sep_y))]
        
       # create a list with the timings for a click event in a trial
        all_clicktimes = []
        # loop over the eventtype list
        for i, trial in enumerate(sep_eventType):
          trial_clickTimes = []
          for k, event in enumerate(trial):
            # if the eventtype is a mouseclick, save the corresponding timestamp
            if event == "MouseClick":
              trial_clickTimes.append(sep_time[i][k])
          all_clicktimes.append(trial_clickTimes)
              
       # create a list with the indices in the interpolated lists that represent
        # mouseclicks
        click_index = []
        # loop over all times in all trials
        for ind, trial in enumerate(all_clicktimes):
          trial_index = []
          for clicktime in trial:
            # find the first entry of the smallest difference between the original
            # click timestamp and the timestamps in the interpolated list
            click_ind = np.where(np.abs(clicktime - int_time[ind]) == np.ndarray.min(np.abs(clicktime - int_time[ind])))[0][0]
            trial_index.append(click_ind)
          click_index.append(trial_index)
            
        # create an event type list that stores information about the eventType
        # of the interpolated datapoint
        new_eventType = []
        for num, trial in enumerate(click_index):
          events = [0 if i in trial else 1 for i in range(len(int_time[num]))]
          new_eventType.append(events)
                
        # add every trial as a feature to the input series list
        for num in range(len(int_x)):  
          # store all features in a list and also add the participant dummy
          features = [int_x[num], int_y[num], new_eventType[num], par_dummy]
          # append the features and the output per trial
          input_series.append(features)
          output_classes.append(output)
          groups.append(par_num)

  return input_series, output_classes, groups

<h3>Create the relevant dataset to use it for classification in the next steps</h3>

In [0]:
# get the relevant dataset (function names):
# create_point_click_data()
# create_drag_drop_data()
# create_drawing_data()
# create_follow_box_data()

# the following steps are the same for all tasks (here the follow_box_task analysis is chosen)
input_series, output_classes, groups = create_follow_box_data()

In [0]:
# resample input series into equal sample size based padding and truncating

# takes a list and a certain length as an input and returns a list of the
# specified length. If the length of the original list is longer than the
# specified list, the original list ist cut to the specified list. If the
# original list is shorter, the list is filled with 0s until the desired length

def trp(item_list, length, filler):
  if length > len(item_list):
    return np.append(item_list[:length], [filler]*(length-len(item_list)), axis=0)
  else:
    return np.asarray(item_list)

# get the longest size of the task trials
size = np.max([len(i[0]) for i in input_series])

resampled_input = []

# resample all other trials to be the same lenght
for trial_input in input_series:
  
  trial_list = []
  
  # resample the input features to match the longest input feature array
  resampled_x = trp(trial_input[0], size, -1)
  resampled_y = trp(trial_input[1], size, -1)
  resampled_event = trp(trial_input[2], size, -1)
  # add x, y and event to resampled trial list
  trial_list.extend([resampled_x, resampled_y, resampled_event])
  
  # get participant info data (one-hot-encoded) and seperate it into the same
  # data format as x, y and events and add to resampled trial list
  for i in trial_input[3]:
    trial_list.append([i] * size)
  
  resampled_input.append(trial_list)
  

In [ ]:
#  get some information about the input characteristics (check to see if everything works)
samples = len(resampled_input)
feature_count = len(resampled_input[0])
timesteps = len(resampled_input[0][0])

X = np.array(resampled_input).reshape((samples, timesteps, feature_count))


print(samples, feature_count, timesteps)
# print(X)

#### Use the resampled dataset for LSTM classification

In [ ]:
# Classification with equal sample length
#----------------------------------------

# Input characteristics
samples = len(resampled_input)
feature_count = len(resampled_input[0])
timesteps = len(resampled_input[0][0])

kfold = GroupKFold(n_splits=5)

# Prepare input and output arrays
X = np.array(resampled_input).reshape((samples, timesteps, feature_count))
y = np.array(output_classes)

# do 5-fold cross validation
for train, test in kfold.split(X, y, groups=groups):

  print("Classification in a CV fold")
  
  X_train, X_test = X[train], X[test]
  y_train, y_test = y[train], y[test]
  
  
  print("-------Shapes-------")
  print(X_train.shape, X_test.shape)


  # Define LSTM model
  model = Sequential()
  model.add(Masking(mask_value=0., input_shape=(timesteps, feature_count)))
  model.add(LSTM(32, dropout=0.2, return_sequences=False, input_shape=(timesteps, feature_count)))
  model.add(Dense(1, activation='sigmoid'))
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  # print(model.summary())

  # Train
  history = model.fit(X_train, y_train, epochs=5, batch_size=64)

  # Evaluate
  scores = model.evaluate(X_test, y_test, verbose=0)
  print("Accuracy: %.2f%%" % (scores[1]*100))
  
  # plt.plot(history.history["loss"])
  # plt.show()

#### Store the results of the analysis

In [0]:
# results:
point_click_results = [50.00, 50.00, 50.00, 50.00, 50.00]
print("PointClick", np.mean(point_click_results), np.std(point_click_results))
results_loading = [54.55, 60.00, 60.00, 50.00, 60.00]
print("Loading", np.mean(results_loading), np.std(results_loading))
results_dragdrop = [50.00, 50.00, 50.00, 50.00, 50.00]
print("DragDrop", np.mean(results_dragdrop), np.std(results_dragdrop))
results_drawing = [50.00, 50.00, 50.00, 50.00, 50.00]
print("Drawing", np.mean(results_drawing), np.std(results_drawing))
results_followBox = [51.82, 48.18, 50.00, 52.00, 49.00]
print("FollowBox", np.mean(results_followBox), np.std(results_followBox))

PointClick 50.0 0.0
Loading 56.910000000000004 4.048752894410821
DragDrop 50.0 0.0
Drawing 50.0 0.0
FollowBox 50.2 1.511608414901161
